## Assignment 1

We first import the necessary libraries:

In [47]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (11, 5)  #set default figure size
from numba import njit, float64
from numba.experimental import jitclass
from scipy.optimize import fsolve, minimize_scalar, bisect
import numpy as np
from interpolation import interp

In [118]:
def maximize(g, a, b, args):
    """
    Maximize the function g over the interval [a, b].

    We use the fact that the maximizer of g on any interval is
    also the minimizer of -g.  The tuple args collects any extra
    arguments to g.

    Returns the maximal value and the maximizer.
    """

    objective = lambda x: -g(x, *args)
    result = minimize_scalar(objective, bounds=(a, b), method='bounded')
    maximizer, maximum = result.x, -result.fun
    return maximizer, maximum

Now, we define an instance of the optimal growth model. 

In [133]:
class ngm():

    def __init__(self, σ=1, β=0.984, δ=0.025, α=1/3, k0 = 1):
        'Parameters:'
        self.σ, self.β, self.δ, self.α, self.k0 = σ, β, δ, α, k0
        
        'Endogenous outcomes and endogenous grid:'
        self.steadystate = self.find_ss()
        self.k0 = 0.75*self.find_ss()
        self.grid = self.create_grid(self.steadystate)
        self.v_initial = self.v_initial()

    def u(self, c):
        '''
        Utility function
        '''
        σ = self.σ

        return c ** (1 - σ) / (1 - σ) if σ!= 1 else np.log(c)

    def u_prime(self, c):
        'Derivative of utility'
        σ = self.σ

        return c ** (-σ)

    def u_prime_inv(self, c):
        'Inverse of derivative of utility'
        σ = self.σ

        return c ** (-1 / σ)

    def f(self, k):
        'Production function'
        α = self.α

        return k ** α

    def f_prime(self, k):
        'Derivative of production function'
        α = self.α

        return α * k ** (α - 1)

    def f_prime_inv(self, k):
        'Inverse of derivative of production function'
        α = self.α 

        return (k / α) ** (1 / (α - 1))
    
    def ss(self, k):
        'Define the steady-state equation'
        β, δ = self.β, self.δ
        f_prime = self.f_prime
        
        return β*(f_prime(k) + (1-δ)) - 1
        
    def find_ss(self):
        'Solve the steady-state equation'
        ss = self.ss
        steady_state = fsolve(ss, 1e-10)
        return steady_state[0]
    
    def create_grid(self, reference):
        'Create a grid as prescribed in the assignment'
        grid = np.linspace(0.5 * reference, 1.5 * reference, 1000)
        return grid
    
    def v_initial(self):
        'Create initialization for value function'
        grid = self.grid
        v_initial = np.empty_like(grid)
        return v_initial      
    
    def state_action_value(self, k, k_new, v_array):
        
        v = lambda x: interp(self.grid, v_array, x) 
        
        u, f = self.u, self.f
        β, δ = self.β, self.δ
        return u(f(k) + (1-δ)*k - k_new) + β*v(k_new) #k_new is the thing we solve for
        # This is the right hand side of the bellman

Next, we implement part two of the code, iteration of the bellman equation

In [136]:
def T(v, ngm):
    """
    The Bellman operator.  Updates the guess of the value function.
    * v is an array representing a guess of the value function

    """
    v_new = np.empty_like(v)
    
    for i, x in enumerate(ngm.grid):
        # Maximize RHS of Bellman equation at state x
        v_new[i] = maximize(ngm.state_action_value, (1-ngm.δ)*x, ngm.f(x) + (1-ngm.δ)*x, (x, v))[1]

    return v_new